<a href="https://colab.research.google.com/github/aditya-kukreja/SAT-Solver/blob/main/Sudoku_Generator_and_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below generates a solved sudoku grid of dimensions side×side



In [1]:
from dpll_algo import *
import sys
import numpy as np


f2= open('DIMACS_Sudoku','w')
f= open('sudoku.txt', 'w')
base  = 2
side  = base**2


def pattern(r,c): 
  return (base*(r%base)+r//base+c)%side


from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]


for line in board: print(line)

[3, 8, 2, 1, 7, 9, 5, 4, 6]
[7, 1, 9, 6, 5, 4, 3, 2, 8]
[5, 6, 4, 8, 3, 2, 7, 9, 1]
[8, 2, 5, 9, 1, 3, 6, 7, 4]
[6, 4, 7, 2, 8, 5, 1, 3, 9]
[1, 9, 3, 4, 6, 7, 8, 5, 2]
[2, 5, 6, 3, 9, 8, 4, 1, 7]
[4, 7, 1, 5, 2, 6, 9, 8, 3]
[9, 3, 8, 7, 4, 1, 2, 6, 5]


The code below deletes random elements from the grid to create a Sudoku Puzzle

---



In [2]:

orig_stdout = sys.stdout

sys.stdout = f

squares = side*side
empties = squares * 1//10
for p in sample(range(squares),empties):
    board[p//side][p%side] = 0

numSize = len(str(side))
for line in board:
    print(*(f"{n or '.':{numSize}}" for n in line),sep=' ')

sys.stdout = orig_stdout
for line in board:
    print(*(f"{n or '.':{numSize}}" for n in line),sep=' ')

f.close()





The below code encodes the sudoku board in the form of a DIMACS file that would be used as the input file for the SAT solver

In [3]:
f3=open('sudoku.txt','r')
literalNo=side**3
x=((side**4)/2)-(side**3/2)+(4*(side**2))
countClues=0
positionList=[]
for i in range(side):
    s=f3.readline().split()
    for j in range(side):
      if s[j] != '.':
        countClues=countClues+1
        positionList.append(i*(side**2)+j*side+int(s[j]))

sys.stdout = f2
       
print("p cnf",literalNo,int(x+countClues))

for i in range(len(positionList)):
  print(positionList[i], 0)

for i in range(side**2):
  for j in range(side):
     print(side*i + j + 1,end=" ")
  print("0")   

for i in range(side**2):
  for j in range(side):
     for k in range(j+1,side):
        print(-1*(side*i+j+1),-1*(side*i+k+1),end=" ")
        print("0")


for i in range(side):
  for j in range(side):
     for k in range(side):
        print((side**2)*i+side*k+j+1,end=" ")
     print("0")        


for i in range(side):
  for j in range(side):
     for k in range(side):
        print(side*i+(side**2)*k+j+1,end=" ")
     print("0")        

for d in range(base):
 for z in range(base): 
   for i in range(side):
     for j in range(base):
       for k in range(base):
          print((side**2)*base*d+(side*base*z)+(side**2)*j+side*k+i+1,end=" ")
     print("0")         




Here we call the DPLL_Solver function to solve the SAT problem and then decode the output file to generate the solved sudoku board

In [5]:
f4=open('solution_SAT.txt','w')
sys.stdout = f4

f5 = open("DIMACS_Sudoku", "r")

literalNum=0
clauseNum=0
while True:
  s=f5.readline()
  s=s.split()
  if s[0][0]=='c' :
    continue
  else:   
    literalNum=int(s[2])
    clauseNum=int(s[3])
    break
clauses = []
literalCount=np.zeros(literalNum)
literalSignCount=np.zeros(literalNum)
literalAssignment=np.full(literalNum, -1)
for i in range(clauseNum):
    s=f5.readline()
    s=s.split()
    a=np.full(literalNum, -1)
    for j in range(len(s)-1):  
      b=int(s[j])
      if b>0:
       literalCount[abs(b)-1]=literalCount[abs(b)-1]+1
       literalSignCount[abs(b)-1]=literalSignCount[abs(b)-1]+1
       a[abs(b)-1]=1
      elif b<0 :
       literalCount[abs(b)-1]=literalCount[abs(b)-1]+1
       literalSignCount[abs(b)-1]=literalSignCount[abs(b)-1]-1
       a[abs(b)-1]=0
      else: 
       break 
    clauses.append(a)


DPLL_Solver(clauses,literalAssignment,literalCount,literalSignCount)

f4.close()

f6=open('solution_SAT.txt','r')
Output=[]
sys.stdout=orig_stdout
x=f6.readline()
x=f6.readline()
x=x.split()
for i in range(side**3):
  if int(x[i])>0:
     if int(x[i])%side==0:
       Output.append(side)
     else:  
       Output.append(int(x[i])%side)

for i in range(side**2):
     print(Output[i],end=" ")
     if i%side==(side-1):
       print(" ")


   


3 8 2 1 7 9 5 4 6  
7 1 9 6 5 4 3 2 8  
5 6 4 8 3 2 7 9 1  
8 2 5 9 1 3 6 7 4  
6 4 7 2 8 5 1 3 9  
1 9 3 4 6 7 8 5 2  
2 5 6 3 9 8 4 1 7  
4 7 1 5 2 6 9 8 3  
9 3 8 7 4 1 2 6 5  
